# Mukh O Mukhosh, the face mask detector

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
from PIL import Image
import os

## dataset read/transform/prepocessing

In [2]:
root_path = "./data"

In [3]:
train_data_path = os.path.join(root_path, "train")

In [4]:
print(train_data_path)

./data/train


In [5]:
train_trasnform = transforms.Compose([
    transforms.Resize(400),
    transforms.ToTensor()
])

In [6]:
train_data = datasets.ImageFolder(train_data_path, train_trasnform)

In [7]:
print(train_data)

Dataset ImageFolder
    Number of datapoints: 1314
    Root Location: ./data/train
    Transforms (if any): Compose(
                             Resize(size=400, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                         )
    Target Transforms (if any): None


In [8]:
train_data.classes

['with_mask', 'without_mask']

In [9]:
train_data.class_to_idx

{'with_mask': 0, 'without_mask': 1}

In [10]:
train_data.imgs

[('./data/train/with_mask/-110603108-gettyimages-533567012.jpg', 0),
 ('./data/train/with_mask/-110695084-mask2.jpg', 0),
 ('./data/train/with_mask/0-with-mask.jpg', 0),
 ('./data/train/with_mask/0000305-62360-halyard-health-pfr-p3-respirator-face-mask-ffp.jpeg',
  0),
 ('./data/train/with_mask/0000444-48296-halyard-health-surgical-mask-teddy-bear-print.jpeg',
  0),
 ('./data/train/with_mask/007563.jpg', 0),
 ('./data/train/with_mask/0779db20580fc7b054ad6b1b7bd9fb35.jpeg', 0),
 ('./data/train/with_mask/1-respirator-mask.jpg', 0),
 ('./data/train/with_mask/10-with-mask.jpg', 0),
 ('./data/train/with_mask/100-with-mask.jpg', 0),
 ('./data/train/with_mask/101-with-mask.jpg', 0),
 ('./data/train/with_mask/103-with-mask.jpg', 0),
 ('./data/train/with_mask/104-with-mask.jpg', 0),
 ('./data/train/with_mask/105-with-mask.jpg', 0),
 ('./data/train/with_mask/106.jpg', 0),
 ('./data/train/with_mask/107-with-mask.jpg', 0),
 ('./data/train/with_mask/108-with-mask.jpg', 0),
 ('./data/train/with_mask

## train image dimension analysis using Pandas

In [11]:
df = pd.DataFrame(train_data.imgs)

In [12]:
df.head()

,0,1
0,./data/train/with_mask/-110603108-gettyimages-...,0
1,./data/train/with_mask/-110695084-mask2.jpg,0
2,./data/train/with_mask/0-with-mask.jpg,0
3,./data/train/with_mask/0000305-62360-halyard-h...,0
4,./data/train/with_mask/0000444-48296-halyard-h...,0


In [13]:
df[0].describe()

count                                        1314
unique                                       1314
top       ./data/train/with_mask/17-with-mask.jpg
freq                                            1
Name: 0, dtype: object

In [14]:
def image_size(img):
    with Image.open(img) as im:
        return im.size

img_sizes = [image_size(img[0]) for img in train_data.imgs]

In [15]:
img_sizes

[(410, 230),
 (976, 549),
 (525, 350),
 (320, 320),
 (1500, 1500),
 (1360, 1360),
 (1279, 720),
 (267, 400),
 (233, 350),
 (320, 428),
 (320, 428),
 (320, 428),
 (324, 341),
 (214, 244),
 (348, 480),
 (320, 433),
 (320, 428),
 (296, 374),
 (640, 640),
 (266, 344),
 (320, 428),
 (320, 428),
 (300, 300),
 (320, 428),
 (295, 400),
 (320, 428),
 (622, 350),
 (320, 428),
 (320, 428),
 (320, 428),
 (320, 428),
 (327, 433),
 (320, 433),
 (312, 351),
 (320, 428),
 (524, 350),
 (340, 417),
 (720, 540),
 (320, 428),
 (320, 428),
 (320, 428),
 (285, 372),
 (259, 194),
 (956, 960),
 (320, 428),
 (184, 274),
 (320, 428),
 (238, 212),
 (320, 428),
 (173, 221),
 (320, 428),
 (467, 350),
 (414, 534),
 (273, 400),
 (205, 246),
 (195, 259),
 (226, 250),
 (206, 244),
 (260, 194),
 (320, 433),
 (201, 289),
 (332, 432),
 (188, 283),
 (320, 433),
 (320, 433),
 (225, 240),
 (257, 196),
 (603, 350),
 (203, 248),
 (318, 442),
 (259, 194),
 (203, 233),
 (225, 225),
 (320, 428),
 (296, 170),
 (189, 267),
 (341, 

In [16]:
df_image_sizes = pd.DataFrame(img_sizes)

In [17]:
df_image_sizes

,0,1
0,410,230
1,976,549
2,525,350
3,320,320
4,1500,1500
5,1360,1360
6,1279,720
7,267,400
8,233,350
9,320,428


In [18]:
df_image_sizes[0].describe()

count    1314.000000
mean      397.927702
std       326.431366
min       110.000000
25%       260.000000
50%       320.000000
75%       349.750000
max      3456.000000
Name: 0, dtype: float64

## data loader

In [19]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)

## convolutional model

In [20]:
class FaceMaskDetectorModel(nn.Module):
    pass